## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.7 MB/s 
     |████████████████████████████████| 182 kB 98.2 MB/s 
     |████████████████████████████████| 7.6 MB 68.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 15.5 MB/s 
     |████████████████████████████████| 132 kB 95.1 MB/s 
     |████████████████████████████████| 212 kB 82.5 MB/s 
     |████████████████████████████████| 127 kB 96.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [ ]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from transformers import TrainingArguments

from InstructATSC.utils import T5Utils
from InstructATSC.Data.data_prep import ModelReadyData

In [ ]:
rest_train_file_path = './InstructATSC/Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './InstructATSC/Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './InstructATSC/Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './InstructATSC/Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [ ]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

 # Get the input text into the required format
bos_instruction = 'Review: '
delim_instruction = ' Aspect: '
eos_insrtuction = ' Sentiment:'
res_tr_df = model_data.create_data_in_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction, delim_instruction, eos_insrtuction)

In [ ]:
# Experimentation
experiment_id = 'laptops'
experiment_name = 'laptops_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5Utils(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

***** Running training *****
  Num examples = 3915
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 980
  Number of trainable parameters = 222903552


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.226152
2,No log,0.218839
3,1.290700,0.252224
4,1.290700,0.262519


***** Running Evaluation *****
  Num examples = 1032
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1032
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Knowle

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation')

# Get prediction labels - OOD Training set
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train')

# Get prediction labels - OOD Training set
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation')

***** Running Prediction *****
  Num examples = 3915
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/t5-base-laptops_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 1032
  Batch size = 16


***** Running Prediction *****
  Num examples = 4713
  Batch size = 16


***** Running Prediction *****
  Num examples = 1328
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] =  id_tr_pred_labels
id_te_df['pred_labels'] =  id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] =  ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain accuracy: ', ood_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(ood_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, ood_te_filename), index = False)

In domain train accuracy:  94.40613026819923
In domain test accuracy:  86.04651162790698
Out of domain accuracy:  82.38702201622247
